In [ ]:
import pandas as pd
df = pd.read_csv("reddit_dataset.csv")

In [ ]:
print(df["target"].value_counts())

target
1    1199
4    1142
2    1085
3    1072
0    1058
Name: count, dtype: int64


In [ ]:
df.isnull().sum()

,0
text,0
title,0
target,0
cleaned_text,0
sentiment,0


In [ ]:
df = df.dropna(subset=["cleaned_text"]).reset_index(drop=True)

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df["title"].tolist(), df["target"].tolist(), test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(df['target'].unique()))


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), np.array(train_labels))).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), np.array(val_labels))).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), np.array(test_labels))).batch(8)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 123s 173ms/step - loss: 1.2978 - accuracy: 0.4598 - val_loss: 0.9866 - val_accuracy: 0.6327
Epoch 2/10
487/487 [==============================] - 74s 152ms/step - loss: 0.8870 - accuracy: 0.6765 - val_loss: 0.9512 - val_accuracy: 0.6663
Epoch 3/10
487/487 [==============================] - 74s 153ms/step - loss: 0.6258 - accuracy: 0.7825 - val_loss: 0.9704 - val_accuracy: 0.7035
Epoch 4/10
487/487 [==============================] - 74s 152ms/step - loss: 0.4254 - accuracy: 0.8609 - val_loss: 1.0921 - val_accuracy: 0.6975
Epoch 5/10
487/487 [==============================] - 74s 152ms/step - loss: 0.3080 - accuracy: 0.9013 - val_loss: 1.2119 - val_accuracy: 0.7047
Epoch 6/10
487/487 [==============================] - 74s 151ms/step - loss: 0.2186 - accuracy: 0.9290 - val_loss: 1.2978 - val_accuracy: 0.7011
Epoch 7/10
487/487 [==============================] - 75s 153ms/step - loss: 0.1588 - accuracy: 0.9501 - val_loss: 1.3364 - val_a

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc}%")

105/105 [==============================] - 7s 44ms/step - loss: 1.4818 - accuracy: 0.7098
Test Accuracy: 0.7098321318626404%


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc}%")

105/105 [==============================] - 9s 81ms/step - loss: 0.9803 - accuracy: 0.7734
Test Accuracy: 0.7733812928199768%
